![imagenes](logo.png)

# Columnas calculadas

![imagenes](im009.png)

Otra transformación común de datos es crear/calcular nuevas variables basadas en las existentes. Por ejemplo, digamos que te sientes más cómodo pensando en la temperatura en grados Celsius (°C) en lugar de grados Fahrenheit (°F). La fórmula para convertir temperaturas de °F a °C es

$$\mbox{temperatura en C}^\circ = \frac{\mbox{temperatura en F}^\circ-32}{1.8}$$

Podemos aplicar esta fórmula a la variable ``temp`` de ``weather`` usando la función ``mutate()`` del paquete ``dplyr``, que toma las variables existentes y las muta para crear otras nuevas.

In [ ]:
library(tidyverse)
library(nycflights13)

In [ ]:
weather <- weather %>% 
  mutate(temp_in_C = (temp - 32) / 1.8)

weather

En este código, *mutamos* el dataframe ``weather`` creando una nueva variable ``temp_in_C = (temp - 32) / 1.8`` y luego sobrescribimos el dataframe original. ¿Por qué sobrescribimos el clima del dataframe, en lugar de asignar el resultado a un nuevo marco de datos como ``weather_new``? Como regla general, mientras no pierdas la información original que puedas necesitar más adelante, es una práctica aceptable sobrescribir los dataframes existentes con los actualizados, como hicimos aquí. Por otro lado, ¿por qué no sobrescribimos la variable ``temp``, sino que creamos una nueva variable llamada ``temp_in_C``? Porque si hiciéramos esto, habríamos borrado la información original contenida en ``temp`` de temperaturas en Fahrenheit que todavía puede ser valiosa para nosotros.

Ahora calculemos las temperaturas medias mensuales tanto en °F como en °C usando ``group_by()`` y ``summarize()``:

In [ ]:
resumen_mensual_temperaturas <- weather %>% 
  group_by(month) %>% 
  summarize(mean_temp_in_F = mean(temp, na.rm = TRUE), 
            mean_temp_in_C = mean(temp_in_C, na.rm = TRUE))

resumen_mensual_temperaturas

Consideremos otro ejemplo. Los pasajeros a menudo se sienten frustrados cuando su vuelo sale tarde, pero no se molestan tanto si, al final, los pilotos pueden recuperar algo de tiempo durante el vuelo. Esto se conoce en la industria de las aerolíneas como **gain** (ganancia), y crearemos esta variable usando la función ``mutate()``:

In [ ]:
flights <- flights %>% 
  mutate(gain = dep_delay - arr_delay)

flights

Echemos un vistazo solo a ``dep_delay``, ``arr_delay`` y la variables resultante ``gain`` para las primeras 5 filas en nuestro dataframe de vuelos actualizado:

In [ ]:
flights %>% 
    select(dep_delay,arr_delay,gain) %>%
    head(5)

El vuelo de la primera fila salió con 2 minutos de retraso pero llegó con 11 minutos de retraso, por lo que su "tiempo ganado en el aire" es una pérdida de 9 minutos, por lo que su ganancia es 2 - 11 = -9. Por otro lado, el vuelo de la cuarta fila partió un minuto antes (``dep_delay`` de -1) pero llegó 18 minutos antes (``arr_delay`` de -18), por lo que su "tiempo ganado en el aire" es −1 − (−18) = −1 + 18 = 17  minutos, por lo que su ganancia es +17.

Veamos algunas estadísticas de resumen de la variable de ganancia considerando varias funciones de resumen a la vez en el mismo código de ``summarize()``:

In [ ]:
gain_summary <- flights %>% 
  summarize(
    min = min(gain, na.rm = TRUE),
    q1 = quantile(gain, 0.25, na.rm = TRUE),
    median = quantile(gain, 0.5, na.rm = TRUE),
    q3 = quantile(gain, 0.75, na.rm = TRUE),
    max = max(gain, na.rm = TRUE),
    mean = mean(gain, na.rm = TRUE),
    sd = sd(gain, na.rm = TRUE),
    missing = sum(is.na(gain))
  )

gain_summary

Vemos, por ejemplo, que la ganancia promedio es de +5 minutos, mientras que la mayor es de +109 minutos.

Recuerde que, dado que la ganancia es una variable numérica, podemos visualizar su distribución usando un histograma.

In [ ]:
ggplot(data = flights, mapping = aes(x = gain)) +
  geom_histogram(color = "white", bins = 20)

El histograma resultante proporciona una perspectiva diferente sobre la variable de ganancia que las estadísticas de resumen que calculamos anteriormente. Por ejemplo, que la mayoría de los valores de ganancia están alrededor de 0.

Para cerrar nuestra discusión sobre la función ``mutate()`` para crear nuevas variables, toma en cuenta que podemos crear varias variables nuevas a la vez en el mismo código ``mutate()``. Además, dentro del mismo código ``mutate()`` podemos referirnos a nuevas variables que acabamos de crear. Como ejemplo, considere el código ``mutate()``:

In [ ]:
flights <- flights %>% 
  mutate(
    gain = dep_delay - arr_delay,
    hours = air_time / 60,
    gain_per_hour = gain / hours
  )

flights

---
---

**Ejercicios.**

**3.6.1.** ¿A qué corresponden los valores positivos de la variable ganancia en vuelos? ¿Qué pasa con los valores negativos? ¿Y qué pasa con un valor cero?

**3.6.2.** ¿Podríamos crear las columnas ``dep_delay`` y ``arr_delay`` simplemente restando ``dep_time`` de ``sched_dep_time`` y de manera similar para las llegadas? Pruebe el código y explique las diferencias entre el resultado y lo que realmente aparece en los vuelos.

**3.6.3.** ¿Qué podemos decir sobre la distribución de la ganancia? Descríbalo en unas pocas oraciones usando la gráfica y los valores del dataframe ``gain_summary``.

---
---